In [4]:
import numpy as np
import pandas as pd
import os
import re

from functools import reduce

In [5]:
DATA_PATH = "./data/prices/tech_companies_15minute.csv"

In [7]:
data = pd.read_csv(DATA_PATH, index_col=0)
display(data.sample(10))
data.info(verbose=True)

,volume,open,close,high,low,date,ticker
269527,1616.0,57.0800,57.1200,57.1200,57.080,2021-06-09 11:00:00,INTC
148553,1028.0,3088.0000,3086.2100,3088.0000,3086.210,2020-09-25 22:30:00,AMZN
68623,562410.0,261.3200,260.9250,261.3850,260.700,2021-06-17 17:45:00,MSFT
265163,826512.0,60.7100,60.8300,60.9800,60.710,2021-02-22 21:00:00,INTC
37777,21977.0,1775.9500,1776.5700,1777.0700,1773.665,2020-11-17 19:00:00,GOOG
59150,929288.0,200.4000,200.5150,200.6700,199.900,2020-10-30 15:45:00,MSFT
95072,805640.0,694.7000,691.5650,695.6640,690.120,2021-03-02 19:15:00,TSLA
267036,548163.0,66.6650,66.5150,66.6954,66.450,2021-04-08 14:15:00,INTC
378558,825410.0,26.3035,26.3177,26.4750,26.126,2020-04-28 13:59:00,TTD
114697,400.0,88.0000,88.0000,88.0000,88.000,2020-06-01 10:34:00,NVDA


<class 'pandas.core.frame.DataFrame'>
Int64Index: 445787 entries, 0 to 445786
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   volume  445787 non-null  float64
 1   open    445787 non-null  float64
 2   close   445787 non-null  float64
 3   high    445787 non-null  float64
 4   low     445787 non-null  float64
 5   date    445787 non-null  object 
 6   ticker  445787 non-null  object 
dtypes: float64(5), object(2)
memory usage: 27.2+ MB


In [8]:
data["ticker"].unique()

array(['AAPL', 'GOOG', 'MSFT', 'TSLA', 'NVDA', 'AMZN', 'FB', 'BABA',
       'CRM', 'INTC', 'AMD', 'PYPL', 'ATVI', 'EA', 'TTD', 'MTCH', 'ZG',
       'YELP'], dtype=object)

In [9]:
close_prices = []
for ticker in data["ticker"].unique():
    close_prices.append(data[data["ticker"] == ticker][["date", "close", "ticker"]].set_index("date"))

In [10]:
current_df = close_prices[0]
for i in range(1, len(close_prices)):
    current_df = current_df.join(close_prices[i], rsuffix=f"_{close_prices[i]['ticker'].iloc[0]}", how="inner")

current_df.sample(10)

,close,ticker,close_GOOG,ticker_GOOG,close_MSFT,ticker_MSFT,close_TSLA,ticker_TSLA,close_NVDA,ticker_NVDA,...,close_EA,ticker_EA,close_TTD,ticker_TTD,close_MTCH,ticker_MTCH,close_ZG,ticker_ZG,close_YELP,ticker_YELP
date,,,,,,,,,,,,,,,,,,,,,
2022-01-24 19:00:00,158.4250,AAPL,2554.350,GOOG,289.590,MSFT,892.0000,TSLA,223.0950,NVDA,...,133.67,EA,62.0800,TTD,115.0700,MTCH,50.9900,ZG,33.31,YELP
2022-01-31 19:45:00,173.4801,AAPL,2699.710,GOOG,308.900,MSFT,934.7700,TSLA,241.8200,NVDA,...,131.56,EA,68.8450,TTD,111.7200,MTCH,49.2175,ZG,34.26,YELP
2021-10-26 19:30:00,149.7811,AAPL,2801.425,GOOG,310.270,MSFT,1019.1900,TSLA,247.9246,NVDA,...,142.81,EA,77.0350,TTD,163.7100,MTCH,96.3450,ZG,39.28,YELP
2021-06-14 18:15:00,130.1001,AAPL,2507.550,GOOG,258.315,MSFT,617.7200,TSLA,179.4713,NVDA,...,148.10,EA,60.4000,TTD,143.3400,MTCH,116.4900,ZG,40.36,YELP
2021-12-20 15:45:00,168.3910,AAPL,2814.760,GOOG,319.360,MSFT,898.1699,TSLA,275.8199,NVDA,...,129.52,EA,88.1800,TTD,127.6300,MTCH,59.8300,ZG,34.62,YELP
2021-07-19 17:15:00,141.7900,AAPL,2577.020,GOOG,275.630,MSFT,636.8000,TSLA,186.8301,NVDA,...,140.99,EA,70.2200,TTD,153.9300,MTCH,105.8000,ZG,35.72,YELP
2021-05-17 13:30:00,126.2700,AAPL,2299.500,GOOG,246.110,MSFT,580.9650,TSLA,140.5850,NVDA,...,137.72,EA,51.6430,TTD,140.5200,MTCH,111.4900,ZG,37.86,YELP
2021-01-19 18:00:00,128.1800,AAPL,1791.210,GOOG,215.350,MSFT,837.3400,TSLA,129.8925,NVDA,...,140.36,EA,77.5050,TTD,146.1003,MTCH,150.2900,ZG,31.05,YELP
2021-12-30 15:00:00,180.2200,AAPL,2941.050,GOOG,341.735,MSFT,1079.2400,TSLA,303.3499,NVDA,...,134.66,EA,94.0367,TTD,132.3800,MTCH,60.5850,ZG,37.13,YELP


In [11]:
current_df = current_df.rename(columns={"close": "close_AAPL"}).drop([col for col in current_df.columns if re.search("ticker.*", col)], axis=1)
current_df.sample(10)

,close_AAPL,close_GOOG,close_MSFT,close_TSLA,close_NVDA,close_AMZN,close_FB,close_BABA,close_CRM,close_INTC,close_AMD,close_PYPL,close_ATVI,close_EA,close_TTD,close_MTCH,close_ZG,close_YELP
date,,,,,,,,,,,,,,,,,,
2021-11-22 15:45:00,164.7425,2971.910,347.6800,1192.6897,342.8900,3601.9100,346.3200,137.3100,301.8000,50.4100,160.1000,186.730,60.8900,127.3900,103.3100,139.17,54.08,36.520
2021-06-07 13:45:00,125.2300,2448.385,252.1100,588.6300,176.1886,3201.1726,330.1800,217.1074,237.7000,57.3201,81.4600,261.530,97.0500,145.2900,57.1020,140.12,110.20,39.170
2021-03-19 16:30:00,120.8890,2043.650,231.4150,643.3200,128.6925,3049.3900,290.4050,238.7500,212.5600,64.6800,79.0150,242.225,91.0500,128.1738,73.1360,151.11,140.60,42.940
2021-07-27 19:45:00,146.7800,2735.920,287.0000,644.7800,192.1000,3628.6400,367.8500,186.0600,243.5200,53.1800,91.1000,300.480,84.0500,141.9400,82.6818,162.17,107.39,38.260
2021-10-19 16:30:00,148.6550,2875.110,308.1200,872.4996,223.0200,3426.1500,341.7350,173.9291,293.9900,54.9800,116.7100,271.320,76.8733,136.4550,81.9800,160.87,86.53,39.675
2022-02-25 14:30:00,161.8900,2639.380,292.6500,787.8649,234.9000,2991.6500,205.2800,105.8400,203.6900,46.4400,116.6701,105.170,81.0300,128.1400,81.2200,110.75,57.14,33.770
2021-11-24 14:30:00,160.5350,2913.010,334.6100,1068.0500,310.8715,3549.4901,334.7506,132.8201,284.0853,48.9650,148.9700,186.645,61.2900,126.5100,101.7200,133.55,54.89,35.690
2022-04-04 15:00:00,177.6500,2872.960,313.7000,1129.6200,272.1800,3333.2500,233.2700,116.6950,218.0800,48.8250,109.0000,120.130,80.8500,128.1200,75.1600,112.54,51.17,35.260
2022-01-27 16:45:00,162.6700,2635.140,305.3290,873.3000,224.0800,2861.3100,301.0050,113.6100,217.1550,48.8350,105.4300,161.800,79.3900,133.2300,62.5400,110.08,47.36,33.050


In [12]:
current_df.corr()

,close_AAPL,close_GOOG,close_MSFT,close_TSLA,close_NVDA,close_AMZN,close_FB,close_BABA,close_CRM,close_INTC,close_AMD,close_PYPL,close_ATVI,close_EA,close_TTD,close_MTCH,close_ZG,close_YELP
close_AAPL,1.000000,0.784577,0.865519,0.791381,0.884586,0.137244,0.018247,-0.889889,0.111600,-0.361209,0.834386,-0.519454,-0.614688,-0.148231,0.281465,-0.485339,-0.783315,0.075912
close_GOOG,0.784577,1.000000,0.956558,0.621618,0.872259,0.404551,0.428427,-0.828416,0.433034,-0.132495,0.705681,-0.124717,-0.510288,0.084528,0.095454,-0.154275,-0.697636,0.490826
close_MSFT,0.865519,0.956558,1.000000,0.767985,0.953578,0.379299,0.350599,-0.853487,0.481617,-0.237502,0.854051,-0.252008,-0.659202,-0.021151,0.266656,-0.220903,-0.754956,0.349211
close_TSLA,0.791381,0.621618,0.767985,1.000000,0.805202,0.182576,-0.013877,-0.685282,0.292525,-0.157423,0.848314,-0.397032,-0.560318,-0.022727,0.406133,-0.234553,-0.537340,0.173914
close_NVDA,0.884586,0.872259,0.953578,0.805202,1.000000,0.316941,0.207462,-0.858019,0.436869,-0.387511,0.924596,-0.426191,-0.762300,-0.187627,0.346149,-0.343804,-0.829054,0.170525
close_AMZN,0.137244,0.404551,0.379299,0.182576,0.316941,1.000000,0.677189,0.025554,0.622156,0.168552,0.232278,0.533738,-0.121017,0.416536,0.345940,0.563070,0.045191,0.391100
close_FB,0.018247,0.428427,0.350599,-0.013877,0.207462,0.677189,1.000000,0.011874,0.639944,0.255266,0.145010,0.690392,-0.152999,0.441480,0.109095,0.564743,0.010415,0.461146
close_BABA,-0.889889,-0.828416,-0.853487,-0.685282,-0.858019,0.025554,0.011874,1.000000,-0.082400,0.368731,-0.749157,0.560742,0.606938,0.243723,-0.049006,0.553287,0.860141,-0.173620
close_CRM,0.111600,0.433034,0.481617,0.292525,0.436869,0.622156,0.639944,-0.082400,1.000000,-0.148030,0.457346,0.330694,-0.478180,0.145511,0.413078,0.450902,-0.160502,0.220367
close_INTC,-0.361209,-0.132495,-0.237502,-0.157423,-0.387511,0.168552,0.255266,0.368731,-0.148030,1.000000,-0.435187,0.622932,0.600471,0.508649,-0.202831,0.494292,0.632147,0.633535


Как видно, акции `AAPL` сильно скоррелированы с акциями `MSFT`, `GOOG`, `TSLA`, `NVDA`, `BABA`, `AMD`, `ATVI`, `ZG`.